In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import dok_matrix
import json
from importlib import import_module
import os
import pickle
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

In [2]:
def zero_based_mapping(data) :
    with open('/opt/ml/movie-recommendation/data/train/zero_mapping.json', 'r') as f:
        dict_data= json.load(f)

    n_user = len(dict_data['user'])
    n_item = len(dict_data['item'])

    data['user']  = data['user'].map(lambda x : dict_data['user'][str(x)])
    data['item']  = data['item'].map(lambda x : dict_data['item'][str(x)])
    
    return data, n_user, n_item

In [5]:
data_dir = '/opt/ml/movie-recommendation/data/train/'

ratings_df = pd.read_csv(data_dir+'train_ratings.csv')
ratings_df

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563
...,...,...,...
5154466,138493,44022,1260209449
5154467,138493,4958,1260209482
5154468,138493,68319,1260209720
5154469,138493,40819,1260209726


In [6]:
def choose_with_prob(p_discard):
    p = random.uniform()
    return False if p < p_discard else True

In [30]:
item_seqs = ratings_df.groupby('user')['item'].agg(list)
item_seqs

user
11        [4643, 170, 531, 616, 2140, 2722, 2313, 2688, ...
14        [8961, 1396, 471, 2105, 1042, 1947, 1269, 2394...
18        [1952, 1283, 3507, 4280, 51084, 593, 318, 356,...
25        [261, 22, 2161, 3255, 372, 1093, 428, 175, 214...
31        [260, 1196, 1210, 7153, 4993, 5952, 1270, 5855...
                                ...                        
138473    [524, 3354, 1025, 6565, 69757, 2085, 32, 55282...
138475    [1639, 1673, 1148, 246, 2019, 1267, 1172, 1235...
138486    [2694, 1994, 2723, 441, 2288, 637, 2013, 2423,...
138492    [2115, 908, 58, 2700, 2599, 1500, 1358, 1288, ...
138493    [3174, 2872, 48780, 2662, 2840, 1566, 2857, 20...
Name: item, Length: 31360, dtype: object

In [34]:
word_frequecy = ratings_df['item'].value_counts()
prob_discard = 1 - np.sqrt(1e-3 / word_frequecy)
n_items = ratings_df['item'].nunique()

In [ ]:
sgns_samples = []
for seq in item_seqs:
    seq = [w for w in seq if choose_with_prob(prob_discard[w])]
    for i in range(len(seq)):
        target = seq[i]
        # generate positive sample
        context_list = []
        j = 0
        while j < len(seq):
            if j != i:
                context_list.append(seq[j])
                sgns_samples.append((target, seq[j], 1))
            j += 1
        # generate negative sample
        for _ in range(len(context_list)):
            neg_idx = random.randrange(0, n_items)
            while neg_idx in context_list:
                neg_idx = random.randrange(0, n_items)
            sgns_samples.append((target, neg_idx, 0))